# Projekt IUM. Etap 1. Iteracja 1.
## Andrii Gamalii, Wiktor Topolski 
### Zadanie “Większość serwisów udostępniających muzykę czy filmiki, poleca coś swoim użytkownikom. Przyszedł czas, abyśmy zaczęli robić tak samo”

# Definicja problemu biznesowego, definicja zadania modelowania i wszystkich założeń, propozycja kryteriów sukcesu

## Problem biznesowy
Polecenie utworów użytkownikom serwisu do strumieniowania muzyki

## Zadanie modelowania
Generacja rekomendacji na podstawie logów sesji użytkowników
## Założenia
Rekomendacje pojawiają się na ekranie startowym  
Rekomendujemy użytkownikowi 10 różnych utworów

## Kryteria sukcesu
Biznesowe: użytkownicy są zadowoleni z rekomendacji  
Analityczne: zarekomendowany utwór nie został pominięty przez użytkownika  

Biznesowe: użytkownicy wybierają rekomendacje  
Analityczne: użytkownik wybierze 1 z 10 zaproponowanych utworów w 50% sytuacji

# Analiza danych

In [12]:
import pandas as pd
import numpy

In [2]:
# Wczytywanie danych z plików

artists_df_orig = pd.read_json("./data/artists.jsonl", lines=True)
sessions_df_orig = pd.read_json("./data/sessions.jsonl", lines=True)
tracks_df_orig = pd.read_json("./data/tracks.jsonl", lines=True)
users_df_orig = pd.read_json("./data/users.jsonl", lines=True)

## Wykonawcy

In [9]:
# Sprawdzenie ile jest rekordów z brakującymi wartościami
print(artists_df_orig.describe(), "\n\n")

artists_df = artists_df_orig.copy(deep=True)
print("Liczba rekordów", len(artists_df))
print("Liczba unikalnych rekordów po id", len(artists_df[['id']].drop_duplicates()))
print("Wyniki: ")
for c in artists_df.columns:
    incomplete_rows = artists_df[artists_df[[c]].isnull().any(axis=1)]
    print(c, len(incomplete_rows))

          id       name                                             genres
count   1667       1667                                               1586
unique  1597       1667                                               1293
top       -1  Lil Nas X  [latin, latin hip hop, reggaeton, reggaeton fl...
freq      71          1                                                 14 


Liczba rekordów 1667
Liczba unikalnych rekordów po id 1597
Wyniki: 
id 0
name 0
genres 81


Tabela wykonawcy zawiera id, informacje o nazwie i gatunkach danego wykonawcy

### Wnioski
Niektóre rekordy mają id = -1, to powoduje, że są nieużyteczne
- id i name: nie mają brakujących wartości
- genres: braki w 81 rekordach, można uzupełnić na podstawie utworów

## Sesje

In [13]:
# Sprawdzenie ile jest rekordów z brakującymi wartościami
print(sessions_df_orig.describe(exclude=[numpy.number]), "\n\n")

sessions_df = sessions_df_orig.copy(deep=True)
print("Liczba rekordów", len(sessions_df))
print("Liczba unikalnych rekordów", len(sessions_df.drop_duplicates()))
print("Wyniki: ")
for c in sessions_df.columns:
    incomplete_rows = sessions_df[sessions_df[[c]].isnull().any(axis=1)]
    print(c, len(incomplete_rows))

                            timestamp                track_id event_type
count                         3173490                 2607738    3014637
unique                            NaN                   22412          4
top                               NaN  7feSbMqJGg9yL0s6ypxy9S       play
freq                              NaN                     175    1668688
mean    2022-11-09 12:10:40.449315840                     NaN        NaN
min        2021-11-09 23:09:17.591000                     NaN        NaN
25%     2022-05-11 01:54:52.632499968                     NaN        NaN
50%     2022-11-08 14:26:41.807500032                     NaN        NaN
75%        2023-05-11 00:00:15.888000                     NaN        NaN
max        2023-11-10 10:53:05.583000                     NaN        NaN 


Liczba rekordów 3173490
Liczba unikalnych rekordów 3173383
Wyniki: 
session_id 0
timestamp 0
user_id 158691
track_id 565752
event_type 158853


Tabela sesje zawiera id sesji, znaczniki czasowe, id użytkownika, id utworu i typ zdarzenia

Brakujących danych jest dużo...
Trochę powtarzających się rekordów (107). Je można bez żadnych konsekwencji usunąć

In [6]:
print("Unikalne wartości event_type", sessions_df['event_type'].unique())

Unikalne wartości event_type ['play' 'advertisment' 'like' 'skip' None]


Niektóre rekordy zawierają wartość 'advertisment' w kolumnie event_type, są one nam nie potrzebne
Możemy bez konsekwencji je usunąć

In [7]:
# liczba rekordów z event_type = 'advertisment'

sessions_df = sessions_df_orig.copy(deep=True)
print(len(sessions_df[sessions_df['event_type'].isin(['advertisment'])]))

407132


Jest tego sporo...

In [8]:
# Sprawdzenie ile jest rekordów z brakującymi wartościami wyłączając takie z event_type = 'advertisment'

sessions_df = sessions_df_orig.copy(deep=True)
# Usuwanie event_type = 'advertisment'
sessions_df = sessions_df[sessions_df['event_type'].isin(['skip', 'like', None, 'play'])]

print("Liczba rekordów", len(sessions_df))
print("Wyniki: ")
for c in sessions_df.columns:
    incomplete_rows = sessions_df[sessions_df[[c]].isnull().any(axis=1)]
    print(c, len(incomplete_rows))

Liczba rekordów 2766358
Wyniki: 
session_id 0
timestamp 0
user_id 138255
track_id 158620
event_type 158853


In [9]:
# liczba rekordów z brakującymi track_id lub event_type

print(len(sessions_df[sessions_df[["track_id", "event_type"]].isnull().any(axis=1)]))

289038


### Wnioski
- user_id: da się wywnioskować na podstawie innego rekordu o takim samym id sesji
- track_id i event_type: te braki powodują bezużyteczność danego rekordu oraz nieciągłość sekwencji akcji użytkownika w sesji, co powoduje, że dane dla takich sesji są mniej użyteczne. Jest to około 10% danych

## Utwory

In [14]:
# Sprawdzenie ile jest rekordów z brakującymi wartościami
print(tracks_df_orig.describe(), "\n\n")

tracks_df = tracks_df_orig.copy(deep=True)
print("Liczba rekordów", len(tracks_df))
print("Liczba unikalnych rekordów", len(tracks_df.drop_duplicates()))
print("Wyniki: ")
for c in tracks_df.columns:
    incomplete_rows = tracks_df[tracks_df[[c]].isnull().any(axis=1)]
    print(c, len(incomplete_rows))

         popularity   duration_ms      explicit  danceability        energy  \
count  21228.000000  2.241200e+04  22412.000000  22412.000000  22412.000000   
mean      61.363953  2.301383e+05      0.166607      0.599096      0.648017   
std        8.038227  7.209562e+04      0.372633      0.158259      0.209577   
min       51.000000  3.062200e+04      0.000000      0.000000      0.000103   
25%       55.000000  1.914930e+05      0.000000      0.495000      0.508000   
50%       60.000000  2.206670e+05      0.000000      0.608000      0.672000   
75%       67.000000  2.562400e+05      0.000000      0.715000      0.816000   
max       99.000000  4.120258e+06      1.000000      0.980000      0.999000   

                key      loudness   speechiness  acousticness  \
count  22412.000000  22412.000000  22412.000000  22412.000000   
mean       5.278824     -7.196227      0.082914      0.267758   
std        3.558813      3.738098      0.089317      0.276111   
min        0.000000    -44.4

Tabela utwory zawiera id utworu, nazwę utworu, popularność utworu, długość utworu, czy jest wulgarna, id artysty, data wydania i parametry opisujące sam utwór

### Wnioski
- bez id nie jesteśmy w stanie użyć danego utworu
- bez name nie możemy przedstawić utworu użytkownikowi
- braki w popularity nie skutkują bezużytecznością danego rekordu
- braki w id_artist powodują problemy z powiązaniem utworów tego samego artysty między sobą 

- wartości atrybutów key i tempo nie są z zakresu [0,1], trzeba je znormalizować  

## Użytkownicy

In [15]:
# Sprawdzenie ile jest rekordów z brakującymi wartościami
print(users_df_orig.describe(), "\n\n")

users_df = users_df_orig.copy(deep=True)
print("Liczba rekordów", len(users_df))
print("Liczba unikalnych rekordów", len(users_df["user_id"].drop_duplicates()))
print("Wyniki: ")
for c in users_df.columns:
    incomplete_rows = users_df[users_df[[c]].isnull().any(axis=1)]
    print(c, len(incomplete_rows))

           user_id  premium_user    id
count  1000.000000    957.000000  55.0
mean    600.500000      0.225705  -1.0
std     288.819436      0.418265   0.0
min     101.000000      0.000000  -1.0
25%     350.750000      0.000000  -1.0
50%     600.500000      0.000000  -1.0
75%     850.250000      0.000000  -1.0
max    1100.000000      1.000000  -1.0 


Liczba rekordów 1000
Liczba unikalnych rekordów 1000
Wyniki: 
user_id 0
name 0
city 0
street 0
favourite_genres 48
premium_user 43
id 945


Tabela użytkownicy zawiera id użytkownika, imię, adres, ulubione gatunki, czy jest użytkownikiem premium oraz pole id, które albo jest puste albo ma wartość -1

In [12]:
# Sprawdzenie ile rekordów mają brakujące wartości w favourite_genres albo premium_user

print(len(users_df[users_df[["favourite_genres", "premium_user"]].isnull().any(axis=1)]))

90


### Wnioski
- w 90 liniach brakuje albo favourite_genres albo premium_user, ale to nie powinno przeszkadzać w realizacji zadania rekomendacji.

- nie wiadomo co znaczy atrybut id, który pojawia się 55 razy i zawsze ma wartość -1.

# Najważniejsze uwagi dla klienta na temat danych

Wykonawcy: 
- potrzebne jest wprowadzenie poprawnych id dla rekordów z id = -1  

Sesje:  
- potrzebne dane z uzupełnionymi track_id i event_type  

Utwory:  
- potrzebne uzupełnienie id i name dla utworów oraz id_artist  

Użytkownicy:  
- potrzebna informacja o znaczeniu atrybutu id  
